# MAGIC Gamma Telescope - TPOT Classification Study

The below gives information about the data set:

The data are MC generated (see below) to simulate registration of high energy gamma particles in a ground-based atmospheric Cherenkov gamma telescope using the imaging technique. Cherenkov gamma telescope observes high energy gamma rays, taking advantage of the radiation emitted by charged particles produced inside the electromagnetic showers initiated by the gammas, and developing in the atmosphere. This Cherenkov radiation (of visible to UV wavelengths) leaks through the atmosphere and gets recorded in the detector, allowing reconstruction of the shower parameters. The available information consists of pulses left by the incoming Cherenkov photons on the photomultiplier tubes, arranged in a plane, the camera. Depending on the energy of the primary gamma, a total of few hundreds to some 10000 Cherenkov photons get collected, in patterns (called the shower image), allowing to discriminate statistically those caused by primary gammas (signal) from the images of hadronic showers initiated by cosmic rays in the upper atmosphere (background). 

Typically, the image of a shower after some pre-processing is an elongated cluster. Its long axis is oriented towards the camera center if the shower axis is parallel to the telescope's optical axis, i.e. if the telescope axis is directed towards a point source. A principal component analysis is performed in the camera plane, which results in a correlation axis and defines an ellipse. If the depositions were distributed as a bivariate Gaussian, this would be an equidensity ellipse. The characteristic parameters of this ellipse (often called Hillas parameters) are among the image parameters that can be used for discrimination. The energy depositions are typically asymmetric along the major axis, and this asymmetry can also be used in discrimination. There are, in addition, further discriminating characteristics, like the extent of the cluster in the image plane, or the total sum of depositions. 

https://archive.ics.uci.edu/ml/datasets/MAGIC+Gamma+Telescope


Attribute Information:

1. fLength: continuous # major axis of ellipse [mm] 
2. fWidth: continuous # minor axis of ellipse [mm] 
3. fSize: continuous # 10-log of sum of content of all pixels [in #phot] 
4. fConc: continuous # ratio of sum of two highest pixels over fSize [ratio] 
5. fConc1: continuous # ratio of highest pixel over fSize [ratio] 
6. fAsym: continuous # distance from highest pixel to center, projected onto major axis [mm] 
7. fM3Long: continuous # 3rd root of third moment along major axis [mm] 
8. fM3Trans: continuous # 3rd root of third moment along minor axis [mm] 
9. fAlpha: continuous # angle of major axis with vector to origin [deg] 
10. fDist: continuous # distance from origin to center of ellipse [mm] 
11. class: g,h # gamma (signal), hadron (background) 

g = gamma (signal): 12332 
h = hadron (background): 6688 

For technical reasons, the number of h events is underestimated. In the real data, the h class represents the majority of the events. 

The simple classification accuracy is not meaningful for this data, since classifying a background event as signal is worse than classifying a signal event as background. For comparison of different classifiers an ROC curve has to be used. The relevant points on this curve are those, where the probability of accepting a background event as signal is below one of the following thresholds: 0.01, 0.02, 0.05, 0.1, 0.2 depending on the required quality of the sample of the accepted events for different experiments.

In [1]:
# Import required libraries
from tpot import TPOTClassifier
from sklearn.cross_validation import train_test_split
import pandas as pd 
import numpy as np

/home/gunny/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Load the data
telescope=pd.read_csv('data/MAGIC Gamma Telescope Data.csv')
telescope.head(5)

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


As can be seen in the above the class object is organized here, and hence for better results, I start with randomly shuffling the data.

In [3]:
telescope_shuffle=telescope.iloc[np.random.permutation(len(telescope))]
telescope_shuffle.head()

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
13166,23.4428,12.8378,2.1590,0.5591,0.3231,11.4572,11.9109,2.6576,27.4814,222.9928,h
456,73.2648,27.0787,3.5292,0.1718,0.0897,28.7515,67.9722,18.8744,6.2195,285.9970,g
13481,34.4471,9.6442,2.5670,0.4688,0.2913,8.5446,-25.1219,11.6323,46.8251,140.0290,h
1433,25.7288,15.0604,2.5723,0.4150,0.2289,1.9492,23.5596,6.7967,27.1409,207.3860,g
8437,26.0372,9.3234,2.4691,0.5739,0.4194,4.8807,21.0046,-4.8025,19.0420,202.1270,g


Above also rearranges the index number and below I basically reset the Index Numbers.

In [4]:
tele=telescope_shuffle.reset_index(drop=True)
tele.head()

,Flength,Fwidth,Fsize,Fconc,Fconc1,Fasym,Fm3long,Fm3trans,Falpha,Fdist,Class
0,23.4428,12.8378,2.1590,0.5591,0.3231,11.4572,11.9109,2.6576,27.4814,222.9928,h
1,73.2648,27.0787,3.5292,0.1718,0.0897,28.7515,67.9722,18.8744,6.2195,285.9970,g
2,34.4471,9.6442,2.5670,0.4688,0.2913,8.5446,-25.1219,11.6323,46.8251,140.0290,h
3,25.7288,15.0604,2.5723,0.4150,0.2289,1.9492,23.5596,6.7967,27.1409,207.3860,g
4,26.0372,9.3234,2.4691,0.5739,0.4194,4.8807,21.0046,-4.8025,19.0420,202.1270,g


# Pre-Processing Data

In [5]:
# Check the Data Type
tele.dtypes

Flength     float64
Fwidth      float64
Fsize       float64
Fconc       float64
Fconc1      float64
Fasym       float64
Fm3long     float64
Fm3trans    float64
Falpha      float64
Fdist       float64
Class        object
dtype: object

Class is a Categorical Variable as can be seen from above. The levels in it are:

In [6]:
for cat in ['Class']:
    print("Levels for catgeory '{0}': {1}".format(cat, tele[cat].unique()))   

Levels for catgeory 'Class': ['h' 'g']


Next, the categorical variable is numerically encoded since TPOT for now cannot handle categorical variables. 'g' is coded as 0 and 'h' as 1.

In [7]:
tele['Class']=tele['Class'].map({'g':0,'h':1})

A check is performed to see if there are any missing values and code then accordingly.

In [8]:
tele = tele.fillna(-999)
pd.isnull(tele).any()

Flength     False
Fwidth      False
Fsize       False
Fconc       False
Fconc1      False
Fasym       False
Fm3long     False
Fm3trans    False
Falpha      False
Fdist       False
Class       False
dtype: bool

In [9]:
tele.shape

(19020, 11)

Finally we store the class labels, which we need to predict, in a separate variable.

In [10]:
tele_class = tele['Class'].values

# Data Analysis using TPOT

To begin our analysis, we need to divide our training data into training and validation sets. The validation set is just to give us an idea of the test set error.

In [11]:
training_indices, validation_indices = training_indices, testing_indices = train_test_split(tele.index, stratify = tele_class, train_size=0.75, test_size=0.25)
training_indices.size, validation_indices.size

(14265, 4755)

After that, we proceed to calling the `fit()`, `score()` and `export()` functions on our training dataset.
An important TPOT parameter to set is the number of generations (via the `generations` kwarg). Since our aim is to just illustrate the use of TPOT, we assume the default setting of 100 generations, whilst bounding the total running time via the `max_time_mins` kwarg (which may, essentially, override the former setting). Further, we enable control for the maximum amount of time allowed for optimization of a single pipeline, via `max_eval_time_mins`.

On a standard laptop with 4GB RAM, each generation takes approximately 5 minutes to run. Thus, for the default value of 100, without the explicit duration bound, the total run time could be roughly around 8 hours.

In [12]:
from config.classifier_models_only import classifier_config_dict
time_allocated = 60

In [13]:
tpot = TPOTClassifier(verbosity=3, max_time_mins=time_allocated, config_dict=classifier_config_dict)
tpot.fit(tele.drop('Class',axis=1).loc[training_indices].values, tele.loc[training_indices,'Class'].values)

11 operators have been imported by TPOT.


Optimization Progress: 100%|██████████| 100/100 [10:12<00:00,  1.68s/pipeline]

_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l1' and loss='hinge' is not supported, Parameters: penalty='l1', loss='hinge', dual=False
_pre_test decorator: _random_mutation_operator: num_test=0 Unsupported set of arguments: The combination of penalty='l2' and loss='hinge' are not supported when dual=False, Parameters: penalty='l2', loss='hinge', dual=False


Optimization Progress: 100%|██████████| 100/100 [10:12<00:00,  1.68s/pipeline]

_pre_test decorator: _random_mutation_operator: num_test=0 Input X must be non-negative


...

_pre_test decorator: _random_mutation_operator: num_test=0 Input X must be non-negative


Optimization Progress: 100%|██████████| 200/200 [33:16<00:00, 18.41s/pipeline]

_pre_test decorator: _random_mutation_operator: num_test=0 Expected n_neighbors <= n_samples,  but n_samples = 50, n_neighbors = 62



60.11772331666667 minutes have elapsed. TPOT will close down.
TPOT closed prematurely. Will use the current best pipeline.


TPOTClassifier(config_dict={'sklearn.naive_bayes.GaussianNB': {}, 'sklearn.naive_bayes.BernoulliNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.naive_bayes.MultinomialNB': {'alpha': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False]}, 'sklearn.tree.DecisionT...0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  ]), 'min_child_weight': range(1, 21), 'nthread': [1]}},
        crossover_rate=0.1, cv=5, disable_update_check=False,
        early_stop=None, generations=1000000, max_eval_time_mins=5,
        max_time_mins=60, memory=None, mutation_rate=0.9, n_jobs=1,
        offspring_size=100, periodic_checkpoint_folder=None,
        population_size=100, random_state=None, scoring=None,
        subsample=1.0, verbosity=3, warm_start=False)

In the above, 7 generations were computed, each giving the training efficiency of fitting model on the training set. As evident, the best pipeline is the one that has the CV score of 85.335%. The model that produces this result is pipeline, consisting of a logistic regression that adds synthetic features to the input data, which then get utilized by a decision tree classifier to form the final predictions.

Next, the test error is computed for validation purposes.

In [13]:
tpot.score(tele.drop('Class',axis=1).loc[validation_indices].values, tele.loc[validation_indices, 'Class'].values)

0.85573080967402737

As can be seen, the test accuracy is 85.573%.

In [14]:
tpot.export('tpot_MAGIC_Gamma_Telescope_pipeline.py')

True

In [ ]:
# %load tpot_MAGIC_Gamma_Telescope_pipeline.py
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator

# NOTE: Make sure that the class is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:0.853347788745
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LogisticRegression(C=10.0, dual=False, penalty="l2")),
    DecisionTreeClassifier(criterion="gini", max_depth=7, min_samples_leaf=5, min_samples_split=7)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)


In [14]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

from src.hypersearch import xgbc_random
from config.xgboost import config_dict

In [16]:
xgbc_random(
    config_dict,
    3600,
    tele.drop('Class',axis=1).loc[training_indices].values, 
    tele.loc[training_indices,'Class'].values,
    tele.drop('Class',axis=1).loc[validation_indices].values, 
    tele.loc[validation_indices, 'Class'].values)

Trace xgbc_random() --- starting new iteration --- time elapsed = 4.291534423828125e-06 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 1, 'learning_rate': 1.0, 'subsample': 0.3, 'min_child_weight': 11, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- best_score updated to 0.8443743427970557
Trace xgbc_random() --- best_kwargs updated to {'n_estimators': 500, 'max_depth': 1, 'learning_rate': 1.0, 'subsample': 0.3, 'min_child_weight': 11, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 1.177706003189087 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.01, 'subsample': 0.25, 'min_child_weight': 15, 'nthread': 1, 'gamma': 1.5, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 1.7515897750854492 secon

Trace xgbc_random() --- starting new iteration --- time elapsed = 70.53637433052063 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'subsample': 0.8500000000000001, 'min_child_weight': 6, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- best_score updated to 0.8898002103049422
Trace xgbc_random() --- best_kwargs updated to {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'subsample': 0.8500000000000001, 'min_child_weight': 6, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 77.90788221359253 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 2, 'learning_rate': 0.01, 'subsample': 0.6500000000000001, 'min_child_weight': 20, 'nthread': 1, 'gamma': 3.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 7

Trace xgbc_random() --- starting new iteration --- time elapsed = 198.37021660804749 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.5, 'subsample': 0.3, 'min_child_weight': 17, 'nthread': 1, 'gamma': 1.0, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapsed = 199.17308354377747 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 5, 'learning_rate': 1.0, 'subsample': 0.8, 'min_child_weight': 16, 'nthread': 1, 'gamma': 1.0, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 199.63368201255798 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 6, 'learning_rate': 1.0, 'subsample': 0.45, 'min_child_weight': 9, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 200.2512617111206 seconds
Trac

Trace xgbc_random() --- starting new iteration --- time elapsed = 298.650178194046 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 0.45, 'min_child_weight': 1, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 309.1830983161926 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 9, 'learning_rate': 1.0, 'subsample': 0.55, 'min_child_weight': 5, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 317.50142550468445 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.5, 'subsample': 0.25, 'min_child_weight': 20, 'nthread': 1, 'gamma': 4.0, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapsed = 317.85420775413513 seconds
Tra

Trace xgbc_random() --- starting new iteration --- time elapsed = 422.2475690841675 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.5, 'subsample': 0.55, 'min_child_weight': 19, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 429.6061460971832 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.01, 'subsample': 0.9500000000000001, 'min_child_weight': 13, 'nthread': 1, 'gamma': 2.5, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 455.0570526123047 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 8, 'learning_rate': 1.0, 'subsample': 0.4, 'min_child_weight': 18, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapse

...

Trace xgbc_random() --- starting new iteration --- time elapsed = 3370.2050490379333 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.01, 'subsample': 0.2, 'min_child_weight': 14, 'nthread': 1, 'gamma': 1.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3370.55242228508 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 2, 'learning_rate': 0.01, 'subsample': 0.7500000000000001, 'min_child_weight': 9, 'nthread': 1, 'gamma': 1.5, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3370.750214099884 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 2, 'learning_rate': 0.1, 'subsample': 0.2, 'min_child_weight': 1, 'nthread': 1, 'gamma': 5.0, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 337

Trace xgbc_random() --- starting new iteration --- time elapsed = 3440.9327206611633 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 7, 'learning_rate': 1.0, 'subsample': 0.8500000000000001, 'min_child_weight': 10, 'nthread': 1, 'gamma': 5.0, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3441.4312314987183 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 1.0, 'subsample': 0.9500000000000001, 'min_child_weight': 8, 'nthread': 1, 'gamma': 4.0, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3442.292892217636 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 1.0, 'subsample': 0.7000000000000001, 'min_child_weight': 7, 'nthread': 1, 'gamma': 1.5, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- ti

Trace xgbc_random() --- starting new iteration --- time elapsed = 3513.2899498939514 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 4, 'learning_rate': 1.0, 'subsample': 0.8, 'min_child_weight': 20, 'nthread': 1, 'gamma': 5.0, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3516.2721076011658 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 8, 'learning_rate': 1.0, 'subsample': 0.4, 'min_child_weight': 15, 'nthread': 1, 'gamma': 4.0, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3516.7297143936157 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 7, 'learning_rate': 1.0, 'subsample': 0.9000000000000001, 'min_child_weight': 12, 'nthread': 1, 'gamma': 4.0, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3526.777055978775 seconds
Tr

Trace xgbc_random() --- starting new iteration --- time elapsed = 3592.5360090732574 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 0.9500000000000001, 'min_child_weight': 18, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3593.6926703453064 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.5, 'subsample': 0.6000000000000001, 'min_child_weight': 19, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3597.0465502738953 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 6, 'learning_rate': 1.0, 'subsample': 0.55, 'min_child_weight': 11, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- duration exceeded --- pr

({'n_estimators': 500,
  'max_depth': 7,
  'learning_rate': 0.1,
  'subsample': 0.8500000000000001,
  'min_child_weight': 6,
  'nthread': 1,
  'gamma': 0.5,
  'colsample_bytree': 0.7999999999999999},
 0.8898002103049422)

In [17]:
xgbc_random(
    config_dict,
    3600,
    tele.drop('Class',axis=1).loc[training_indices].values, 
    tele.loc[training_indices,'Class'].values,
    tele.drop('Class',axis=1).loc[validation_indices].values, 
    tele.loc[validation_indices, 'Class'].values)

Trace xgbc_random() --- starting new iteration --- time elapsed = 2.1457672119140625e-06 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 6, 'learning_rate': 0.01, 'subsample': 0.7000000000000001, 'min_child_weight': 2, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- best_score updated to 0.8569926393270242
Trace xgbc_random() --- best_kwargs updated to {'n_estimators': 50, 'max_depth': 6, 'learning_rate': 0.01, 'subsample': 0.7000000000000001, 'min_child_weight': 2, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 0.4846665859222412 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 9, 'learning_rate': 0.5, 'subsample': 0.2, 'min_child_weight': 5, 'nthread': 1, 'gamma': 1.0, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 0.9001574516296387 seconds
Trace xgbc_rando

Trace xgbc_random() --- starting new iteration --- time elapsed = 55.179139614105225 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 0.1, 'subsample': 0.8, 'min_child_weight': 7, 'nthread': 1, 'gamma': 2.5, 'colsample_bytree': 0.5}
Trace xgbc_random() --- best_score updated to 0.8853838065194533
Trace xgbc_random() --- best_kwargs updated to {'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 0.1, 'subsample': 0.8, 'min_child_weight': 7, 'nthread': 1, 'gamma': 2.5, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapsed = 60.622464656829834 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.5, 'subsample': 0.6000000000000001, 'min_child_weight': 7, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 62.53682994842529 seconds
Trace xgbc_random(

Trace xgbc_random() --- starting new iteration --- time elapsed = 160.67985272407532 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 0.001, 'subsample': 0.7500000000000001, 'min_child_weight': 2, 'nthread': 1, 'gamma': 1.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 162.5452263355255 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.5, 'subsample': 0.8, 'min_child_weight': 1, 'nthread': 1, 'gamma': 0.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 164.4921407699585 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.1, 'subsample': 0.6000000000000001, 'min_child_weight': 7, 'nthread': 1, 'gamma': 4.0, 'colsample_bytree': 0.6}
Trace xgbc_random() --- best_score updated to 0.8

Trace xgbc_random() --- starting new iteration --- time elapsed = 235.33183360099792 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.5, 'subsample': 0.8, 'min_child_weight': 14, 'nthread': 1, 'gamma': 2.5, 'colsample_bytree': 0.8999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 236.4884970188141 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 6, 'learning_rate': 0.5, 'subsample': 0.8500000000000001, 'min_child_weight': 10, 'nthread': 1, 'gamma': 0.0, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapsed = 236.86331701278687 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.001, 'subsample': 0.2, 'min_child_weight': 7, 'nthread': 1, 'gamma': 4.5, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed 

Trace xgbc_random() --- starting new iteration --- time elapsed = 316.9763729572296 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.01, 'subsample': 0.45, 'min_child_weight': 9, 'nthread': 1, 'gamma': 0.0, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapsed = 325.0303866863251 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 9, 'learning_rate': 0.01, 'subsample': 0.25, 'min_child_weight': 11, 'nthread': 1, 'gamma': 3.5, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 333.7767102718353 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 9, 'learning_rate': 1.0, 'subsample': 0.35000000000000003, 'min_child_weight': 18, 'nthread': 1, 'gamma': 5.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 334.573001

...

Trace xgbc_random() --- starting new iteration --- time elapsed = 3234.8580162525177 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 2, 'learning_rate': 0.5, 'subsample': 0.8500000000000001, 'min_child_weight': 11, 'nthread': 1, 'gamma': 5.0, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3239.0949370861053 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 2, 'learning_rate': 1.0, 'subsample': 0.1, 'min_child_weight': 7, 'nthread': 1, 'gamma': 0.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3240.102037668228 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 8, 'learning_rate': 0.5, 'subsample': 0.7000000000000001, 'min_child_weight': 10, 'nthread': 1, 'gamma': 2.0, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration ---

Trace xgbc_random() --- starting new iteration --- time elapsed = 3349.8522753715515 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 1.0, 'subsample': 1.0, 'min_child_weight': 9, 'nthread': 1, 'gamma': 4.0, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3357.556489467621 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.01, 'subsample': 0.8500000000000001, 'min_child_weight': 19, 'nthread': 1, 'gamma': 3.5, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3362.883410692215 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 1, 'learning_rate': 1.0, 'subsample': 0.9500000000000001, 'min_child_weight': 11, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.6}
Trace xgbc_random() --- starting new iteration --- time elapsed = 

Trace xgbc_random() --- starting new iteration --- time elapsed = 3448.35098195076 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 1.0, 'subsample': 0.6500000000000001, 'min_child_weight': 14, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3456.7570436000824 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 10, 'learning_rate': 0.01, 'subsample': 0.7000000000000001, 'min_child_weight': 7, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3473.709998846054 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 100, 'max_depth': 8, 'learning_rate': 1.0, 'subsample': 0.6000000000000001, 'min_child_weight': 13, 'nthread': 1, 'gamma': 0.5, 'colsample_bytree': 0.7}
Trace xgbc_random() --- starting new iteration --- time elapsed

Trace xgbc_random() --- starting new iteration --- time elapsed = 3536.1715824604034 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 50, 'max_depth': 3, 'learning_rate': 0.5, 'subsample': 0.15000000000000002, 'min_child_weight': 1, 'nthread': 1, 'gamma': 0.0, 'colsample_bytree': 0.5}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3536.3110818862915 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 500, 'max_depth': 10, 'learning_rate': 0.01, 'subsample': 0.6000000000000001, 'min_child_weight': 3, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.7999999999999999}
Trace xgbc_random() --- starting new iteration --- time elapsed = 3547.1814131736755 seconds
Trace xgbc_random() --- trying hyperparameters = {'n_estimators': 1000, 'max_depth': 8, 'learning_rate': 0.01, 'subsample': 0.35000000000000003, 'min_child_weight': 10, 'nthread': 1, 'gamma': 3.0, 'colsample_bytree': 0.9999999999999999}
Trace xgbc_random() --- startin

({'n_estimators': 1000,
  'max_depth': 4,
  'learning_rate': 0.1,
  'subsample': 0.9500000000000001,
  'min_child_weight': 1,
  'nthread': 1,
  'gamma': 0.0,
  'colsample_bytree': 0.8999999999999999},
 0.8912723449001052)